
# **CHATBOT USING STATIC DATASET**

**Importing Useful** **Libraries** 

In [10]:
#for importing and manuplating data
import numpy as np 
import pandas as pd 
import string
import csv

#for language understanding, fitting model
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline

**Importing Dataset From Drive**

In [11]:
#original data is in the form of google sheets
#converting sheets to csv file

sheet_url = 'https://docs.google.com/spreadsheets/d/1KqvA5O9hH62UQWYpn2DVA5Pf4Kofp4gk51hZLVSuJpw/edit#gid=0'

csv_file = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

#reading dataset into variable 'df'
df = pd.read_csv(csv_file)

In [12]:
#reviewing data

df

,User,Chatbot
0,Drug Reaction,"stop irritation,consult nearest hospital,stop ..."
1,Malaria,"Consult nearest hospital,avoid oily food,avoid..."
2,Allergy,"apply calamine,cover area with bandage,use ice..."
3,Hypothyroidism,"reduce stress,exercise,exercise,get proper sleep"
4,Psoriasis,"wash hands with warm soapy water,stop bleeding..."
...,...,...
139,book an appointment for me,"Sorry, I can't help you with this"
140,name a hospital near me,"Sorry, I can't help you with this"
141,do you know my location?,"Sorry, I can't help you with this"
142,can you help me?,"Yes, I think I can. Please enter a disease and..."


**Working on Healthcare Dataset**

In [13]:
#checking for empty columns

df['Chatbot'].isnull().sum()

32

In [14]:
#removing rows with empty columns

df = df.dropna()


### Dataset is now ready for use

In [15]:
#connecting to user inputs dataset

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
#input function

def user_input():
  a = "Hello there! I am Reyana, your healthcare assistant. Please enter your details to continue."
  b = "Enter your "
  fields = ['name', 'email', 'age', 'current location']                       #fields of input
  result = []

  print(a)
  for i in range(len(fields)):
    print(b+fields[i])
    inp = input()
    result.append(inp)

  #input to a csv file
  filename = '/content/drive/MyDrive/Healthcare Chatbot 1/User_dataset.csv' 
    
  with open(filename, 'a+', newline='\n', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(result)
    f.close()


In [17]:
#calling input function

user_input()

Hello there! I am Reyana, your healthcare assistant. Please enter your details to continue.
Enter your name
User1
Enter your email
abc@gmail.com
Enter your age
21
Enter your current location
Delhi


In [18]:
#display csv file head

inputs = pd.read_csv('/content/drive/MyDrive/Healthcare Chatbot 1/User_dataset.csv')

inputs.head()

,name,email,age,current location
0,Aditi,aditisharma100201@gmail.com,20,Noida
1,User1,abc@gmail.com,21,Delhi


***Decision Tree Classifier***

In [19]:
#function to remove punctuation
#converts the input to lowercase

def cleaner(x):
  return [a for a in (''.join([a for a in x if a not in string.punctuation])).lower().split()]

In [20]:
#Creating a pipeline to facilitate conversation
#Using Decision Tree for fitting data

Pipe = Pipeline([
    ('bow',CountVectorizer(analyzer=cleaner)),
    ('tfidf',TfidfTransformer()),
    ('classifier',DecisionTreeClassifier())
])

In [21]:
#Fitting our dataset in the pipeline

Pipe.fit(df['User'],df['Chatbot'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function cleaner at 0x7f24975d98c0>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tok...
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
     

***Talking to chatbot***

In [22]:
input = ""
input = 'MALARIA'

In [23]:
output = Pipe.predict([input])[0]

In [24]:
output

'Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out'

Random Tests

In [25]:
Pipe.predict(['can you help me?'])[0]

'Yes, I think I can. Please enter a disease and I will suggest you some precautions for it.'

In [26]:
Pipe.predict(['Who are you?'])[0]

'Hi, I am Reyana, your healthcare assistant'

In [27]:
Pipe.predict(['Diabetes'])[0]

'have balanced diet,exercise.consult doctor,follow up'